In [37]:
import os
#import ipdb

import pandas as pd
import matplotlib as mpl
from matplotlib import lines
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import seaborn as sns

EXP_NAME = "logs/clients"
RESULTS_DIR = os.path.join( os.getcwd(),'results')

headers = ['time', 'start_transfer', 'download','status','size']

#Read input files from the directory and returns DataTable object in a combined list
def read_clients():
    #Get list of nodes(files) with results
    INPUT_DIR = os.path.join( os.getcwd(), EXP_NAME)
    dirs = [f for f in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR, f))]
    #Load results from each node
    nodes_results = {}
    for sub_dir in dirs:
        client_nodes = [f for f in os.listdir(INPUT_DIR+'/'+sub_dir) if os.path.isdir(os.path.join(INPUT_DIR+'/'+sub_dir, f))]
        #print(client_nodes)
        for node in client_nodes:
            tunnel_nodes = os.listdir(INPUT_DIR+'/'+sub_dir+'/'+node)
            for tunnel in tunnel_nodes:                
                file = os.path.join(INPUT_DIR+'/'+sub_dir+'/'+node,tunnel)
                try:
                    temp = pd.read_csv(file,sep=',', header = None)
                    temp.columns = headers
                    temp = temp.query('status == 200 & size >0')
                    temp = temp.set_index(pd.DatetimeIndex(temp.time))
                    temp['proxy'] = tunnel
                    if node in nodes_results:
                        nodes_results[node] = pd.concat([nodes_results[node], temp])
                        #print('Coming:',node,':',tunnel,':',len(nodes_results[node]))
                    else:
                        nodes_results[node] = temp
                except Exception as e:
                    print(node, 'EXception', e)
            #print(nodes_results[node].info())
    return nodes_results

client_data = read_clients()
for client in client_data:
    df = client_data[client]
    df['download_time']= df['download'] - df['start_transfer']
    proxies = df.proxy.unique()
    for proxy in proxies:
        df_proxy = df.query('proxy == "'+proxy+'"')
        print(client,':',proxy,':',df_proxy['size'].mean(),':',df_proxy.download_time.mean())
    

1mb : 5126 : 125000.0 : 0.536235809820315
1mb : 10473 : 125000.0 : 0.298983509707025
1mb : 11252 : 125000.0 : 0.2692192416095599
1mb : 7193 : 125000.0 : 0.0822205302284707
1mb : 57064 : 125000.0 : 0.04357070993207276
youtube : 5126 : 503359.42287849134 : 3.7110015974025967
youtube : 10473 : 503467.6089641786 : 2.2606202383977423
youtube : 11252 : 502853.2528674784 : 1.8971438501852795
youtube : 7193 : 503409.73989455187 : 1.3579076592267172
youtube : 57064 : 501408.90016731736 : 1.3277825369027698
10mb : 5126 : 1250000.0 : 6.647648807186036
10mb : 10473 : 1250000.0 : 2.0139322023641504
10mb : 11252 : 1250000.0 : 1.4622850015814435
10mb : 7193 : 1249671.8288525743 : 0.5241259550166932
10mb : 57064 : 1250000.0 : 0.3684814232881731
bbc : 5126 : 203172.75004450773 : 0.87853686701086
bbc : 10473 : 203180.00688073394 : 0.621518829216656
bbc : 11252 : 203204.7060691602 : 0.36002063955539876
bbc : 7193 : 203153.7596694796 : 0.2323777317158933
bbc : 57064 : 203125.08131868133 : 0.24868521062271